In [1]:
import pandas as pd
import os
import glob
import re
from natasha import MorphVocab, AddrExtractor

In [2]:
data_path = '../data/'

In [3]:
exp_files = glob.glob(os.path.join(data_path + 'preprocessed/Expenses', '*.feather'))
exp_dfs = []
for filename in exp_files:
    exp_dfs.append(pd.read_feather(filename))
expenses_data = pd.concat(exp_dfs, axis=0, ignore_index=True)
print(expenses_data.shape)

(23150, 10)


In [4]:
unqie_values = expenses_data[expenses_data['Год сметы'] == '2020'].apply(pd.Series.nunique)
unqie_values

Год сметы                        1
Смета                           13
Подразделение БР                20
Код статьи                       1
Код вида расходов                5
Наименование вида расходов       5
Направление расходов           249
Отнесено                      2620
Конечный остаток              2620
Дата проведения                213
dtype: int64

In [5]:
expenses_data = expenses_data[expenses_data.columns.difference(unqie_values[unqie_values == 1].index)]

In [6]:
expenses_data[expenses_data['Конечный остаток'] != expenses_data['Отнесено']].shape[0]

0

In [7]:
expenses_data.drop(columns=['Конечный остаток', 'Смета'], inplace=True)

In [8]:
expenses_data['Код вида расходов'].apply(len).value_counts()

Код вида расходов
15    23150
Name: count, dtype: int64

In [9]:
expenses_data['Код вида расходов'].apply(lambda c: c[-4:] == '0000').value_counts()

Код вида расходов
True    23150
Name: count, dtype: int64

In [10]:
expenses_data['Код вида расходов'] = expenses_data['Код вида расходов'].apply(lambda c: c[:-4])

In [11]:
expenses_data['Направление расходов'] = expenses_data.apply(
    lambda exp: exp['Направление расходов']\
        .replace(exp['Наименование вида расходов'], '')\
        .replace(exp['Код вида расходов'], '')\
        .replace('ст.' + exp['Код вида расходов'][:-3], '')\
        .replace(exp['Код вида расходов'][:-3], '')\
        .replace('Расходы прошлого года', '')\
        .replace('зд', '').replace('Здание', '')\
        .replace('Гараж', '').replace('Пристройка', '')
        .strip(), axis=1
)

In [12]:
addr_extractor = AddrExtractor(MorphVocab())

expenses_data['Адрес объекта'] = expenses_data['Направление расходов'].apply(
    lambda x: [i.fact.as_json for i in addr_extractor(x)]
)

In [13]:
expenses_data[expenses_data['Адрес объекта'].apply(lambda l: l == [])].shape[0] / 23150

0.23460043196544275

In [19]:
expenses_data.loc[expenses_data['Адрес объекта'].apply(lambda l: l == []), 'Код вида расходов'].value_counts()

Код вида расходов
132.0113.00    2419
132.0111.00    1267
132.0112.00     921
134.1100.00     420
134.1000.03     128
134.1100.03      75
132.0114.00      43
134.1000.02      43
134.1100.01      38
134.1000.01      38
134.1100.02      36
134.1100.05       2
134.1000.05       1
Name: count, dtype: int64

In [26]:
expenses_data.loc[expenses_data['Код вида расходов'] == '132.0113.00', 'Направление расходов'].value_counts()

Направление расходов
Другие расходы.Оплата водоснабжения                                           120
                                                                               81
Оплата водоотведения                                                           78
Оплата водоснабжения                                                           73
Другие расходы.Оплата водоотведения                                            63
                                                                             ... 
. ул.Тихоокеанская, 175А                                                        1
Оплата за горячее водоснабжение, ул Пролетарская 17 (гараж)  - Отд Магадан      1
гараж, ул. Пролетарская, 8                                                      1
, признанные после отчетной даты..Вилючинск, м-н Центральный, 5                 1
.  ул. Южная                                                                    1
Name: count, Length: 276, dtype: int64

In [27]:
expenses_data = expenses_data[~expenses_data['Адрес объекта'].apply(lambda l: l == [])].reset_index(drop=True)

In [62]:
expenses_data['Адрес объекта'].apply(
    lambda x: [addr_part for addr_part in x if len(addr_part) > 1]
)

0               [{'value': 'Мордовцева', 'type': 'улица'}]
1              [{'value': 'Светланская', 'type': 'улица'}]
2              [{'value': 'Светланская', 'type': 'улица'}]
3              [{'value': 'Светланская', 'type': 'улица'}]
4              [{'value': 'Светланская', 'type': 'улица'}]
                               ...                        
17714    [{'value': 'Хабаровск', 'type': 'город'}, {'va...
17715    [{'value': 'Хабаровск', 'type': 'город'}, {'va...
17716    [{'value': 'Хабаровск', 'type': 'город'}, {'va...
17717    [{'value': 'Хабаровск', 'type': 'город'}, {'va...
17718    [{'value': 'Хабаровск', 'type': 'город'}, {'va...
Name: Адрес объекта, Length: 17719, dtype: object

: 

In [ ]:
# сделать дубль колонки, заполнение вниз
# по исходной колонке удалить Итого